# College Enrollment System with Google ADK Agents

This notebook demonstrates a complete AI agent system for course enrollment management

In [3]:
# Import required libraries
import sqlite3
import uuid
import os
from google.genai import types
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner, InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.apps.app import App, ResumabilityConfig
from google.adk.tools.function_tool import FunctionTool

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [4]:
# Set up Google API Key
os.environ["GOOGLE_API_KEY"] = "..."
print("✅ API Key configured")

✅ API Key configured


In [5]:
# Clean up any existing database to start fresh
if os.path.exists("student_courses.db"):
    print("student_courses.db exists, clearing...")
    os.remove("student_courses.db")
print("✅ Cleaned up old database files")

student_courses.db exists, clearing...
✅ Cleaned up old database files


## Step 1: Create and Populate Database

We'll create two tables:
- **courses**: Contains course information (name, class level, description, ID)
- **enrollments**: Contains student enrollment records

In [7]:
# Create the database and tables
conn = sqlite3.connect('student_courses.db')
cursor = conn.cursor()

# Create enrollments table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS enrollments (
        student_first_name TEXT NOT NULL,
        student_last_name TEXT NOT NULL,
        student_id INTEGER NOT NULL,
        course_name TEXT NOT NULL,
        course_id INTEGER NOT NULL
    )
''')

# Create courses table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS courses (
        course_name TEXT NOT NULL,
        class TEXT NOT NULL,
        course_description TEXT NOT NULL,
        course_id INTEGER NOT NULL
    )
''')

# Insert course data
ls_courses = [
    ('Introduction To Python', 'freshman', 'Basic concepts of Python and Programming Basics', 101),
    ('Introduction To JavaScript', 'freshman', 'Basic concepts of JavaScript and Programming Basics', 102),
    ('Web Development - Frameworks and Architecture', 'sophomore', 'Main Concepts in Web Development', 201),
    ('Statistics 101', 'freshman', 'Introduction to descriptive statistics', 103), 
    ('Statistics 201', 'sophomore', 'Advanced statistical techniques and their applications', 202),
    ('Introduction To Machine Learning', 'sophomore', 'Machine learning basics and its applications', 203),
    ('Introduction To Neural Networks', 'sophomore', 'Neural network and deep learning basics and its applications', 204)
]

cursor.executemany('''
    INSERT INTO courses 
    (course_name, class, course_description, course_id)
    VALUES (?, ?, ?, ?)
''', ls_courses)

conn.commit()
conn.close()

print("✅ Database created and populated with course data")

✅ Database created and populated with course data


## Step 2: Define Tool Functions

These functions will be wrapped as tools that our agents can use to interact with the database.

In [9]:
def list_available_courses(topic: str = "") -> str:
    """
    Lists all available courses or filters by topic.
    
    Args:
        topic: Optional topic to filter courses by (searches in course name and description)
    
    Returns:
        A formatted string listing the available courses
    """
    if topic == "":
        with sqlite3.connect('student_courses.db') as conn:
            cursor = conn.cursor()
            cursor.execute('SELECT * FROM courses')
            res = cursor.fetchall()
        
        return_str = 'We are offering the following courses: \n'
        for element in res:
            element_str = f' - course: {element[0]}, class: {element[1]}, description: {element[2]}, course id: {element[3]} \n'
            return_str = return_str + element_str
        return return_str
    else:
        with sqlite3.connect('student_courses.db') as conn:
            cursor = conn.cursor()
            cursor.execute(f'''
                SELECT * FROM courses
                WHERE LOWER(course_name) LIKE '%{topic.lower()}%'
                OR LOWER(course_description) LIKE '%{topic.lower()}%'
            ''')
            res = cursor.fetchall()

        if len(res) == 0:
            return f'We currently do not have courses in your topic of choice: {topic}.'
        else:
            return_str = f'We are offering the following courses on {topic}: \n'
            for element in res:
                element_str = f' - course: {element[0]}, class: {element[1]}, description: {element[2]}, course id: {element[3]} \n'
                return_str = return_str + element_str
            return return_str


def enroll_student(
    first_name: str, 
    last_name: str, 
    student_id: int, 
    course_id: int) -> str:
    """
    Enrolls a student in a course by inserting a record into the enrollments table.
    
    Args:
        first_name: Student's first name
        last_name: Student's last name
        student_id: Student's unique ID number
        course_id: The ID of the course to enroll in
    
    Returns:
        A confirmation message about the enrollment
    """
    with sqlite3.connect('student_courses.db') as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT course_name FROM courses WHERE course_id = ?', (course_id,))
        result = cursor.fetchone()
        
        if result is None:
            return f"Error: Course with ID {course_id} does not exist."
        
        course_name = result[0]
        
        # Check if student is already enrolled
        cursor.execute('''
            SELECT * FROM enrollments 
            WHERE student_id = ? AND course_id = ?
        ''', (student_id, course_id))
        
        if cursor.fetchone() is not None:
            return f"Student {first_name} {last_name} (ID: {student_id}) is already enrolled in {course_name}."
        
        # Insert the enrollment
        cursor.execute('''
            INSERT INTO enrollments 
            (student_first_name, student_last_name, student_id, course_name, course_id)
            VALUES (?, ?, ?, ?, ?)
        ''', (first_name, last_name, student_id, course_name, course_id))
        
        conn.commit()
    
    return f"✅ Successfully enrolled {first_name} {last_name} (ID: {student_id}) in {course_name} (Course ID: {course_id})."


def get_students_in_course(course_id: int) -> str:
    """
    Retrieves all students enrolled in a specific course.
    
    Args:
        course_id: The ID of the course
    
    Returns:
        A formatted string listing all enrolled students
    """
    with sqlite3.connect('student_courses.db') as conn:
        cursor = conn.cursor()
        
        # Get course name
        cursor.execute('SELECT course_name FROM courses WHERE course_id = ?', (course_id,))
        course_result = cursor.fetchone()
        
        if course_result is None:
            return f"Error: Course with ID {course_id} does not exist."
        
        course_name = course_result[0]
        
        # Get enrolled students
        cursor.execute('''
            SELECT student_first_name, student_last_name, student_id 
            FROM enrollments 
            WHERE course_id = ?
        ''', (course_id,))
        
        results = cursor.fetchall()
    
    if len(results) == 0:
        return f"No students are currently enrolled in {course_name} (Course ID: {course_id})."
    
    return_str = f"Students enrolled in {course_name} (Course ID: {course_id}):\n"
    for student in results:
        return_str += f" - {student[0]} {student[1]} (ID: {student[2]})\n"
    
    return return_str


def get_student_enrollments(student_id: int) -> str:
    """
    Retrieves all courses a specific student is enrolled in.
    
    Args:
        student_id: The student's unique ID number
    
    Returns:
        A formatted string listing all courses the student is enrolled in
    """
    with sqlite3.connect('student_courses.db') as conn:
        cursor = conn.cursor()
        
        cursor.execute('''
            SELECT student_first_name, student_last_name, course_name, course_id 
            FROM enrollments 
            WHERE student_id = ?
        ''', (student_id,))
        
        results = cursor.fetchall()
    
    if len(results) == 0:
        return f"No enrollments found for student ID {student_id}."
    
    student_name = f"{results[0][0]} {results[0][1]}"
    return_str = f"Enrollments for {student_name} (ID: {student_id}):\n"
    for enrollment in results:
        return_str += f" - {enrollment[2]} (Course ID: {enrollment[3]})\n"
    
    return return_str


def search_students_by_name(name: str) -> str:
    """
    Searches for students by first or last name.
    
    Args:
        name: The name (or partial name) to search for
    
    Returns:
        A formatted string listing matching students and their enrollments
    """
    with sqlite3.connect('student_courses.db') as conn:
        cursor = conn.cursor()
        
        cursor.execute(f'''
            SELECT DISTINCT student_first_name, student_last_name, student_id 
            FROM enrollments 
            WHERE LOWER(student_first_name) LIKE '%{name.lower()}%'
            OR LOWER(student_last_name) LIKE '%{name.lower()}%'
        ''')
        
        results = cursor.fetchall()
    
    if len(results) == 0:
        return f"No students found matching '{name}'."
    
    return_str = f"Students matching '{name}':\n"
    for student in results:
        return_str += f" - {student[0]} {student[1]} (ID: {student[2]})\n"
    
    return return_str

print("✅ All tool functions defined")

✅ All tool functions defined


## Step 3: Helper function

In [107]:
# Define helper functions that will be reused throughout the notebook
async def run_session(
    runner_instance: Runner,
    user_queries: list[str] | str = None,
    session_name: str = "default",
):
    print(f"\n ### Session: {session_name}")

    # Get app name from the Runner
    app_name = runner_instance.app_name

    # Attempt to create a new session or retrieve an existing one
    try:
        session = await session_service.create_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )
    except:
        session = await session_service.get_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )

    # Process queries if provided
    if user_queries:
        # Convert single query to list for uniform processing
        if type(user_queries) == str:
            user_queries = [user_queries]

        # Process each query in the list sequentially
        for query in user_queries:
            print(f"\nUser > {query}")

            # Convert the query string to the ADK Content format
            query = types.Content(role="user", parts=[types.Part(text=query)])

            # Stream the agent's response asynchronously
            async for event in runner_instance.run_async(
                user_id=USER_ID, session_id=session.id, new_message=query
            ):
                # Check if the event contains valid content
                if event.content and event.content.parts:
                    # Filter out empty or "None" responses before printing
                    if (
                        event.content.parts[0].text != "None"
                        and event.content.parts[0].text
                    ):
                        print(f"{MODEL_NAME} > ", event.content.parts[0].text)
    else:
        print("No queries!")


print("✅ Helper functions defined.")

✅ Helper functions defined.


## Step 4: Create Specialized Agents

We'll create three agents:
1. **Course Information Agent** - Helps students discover courses
2. **Enrollment Agent** - Handles interactive enrollment process
3. **Administrator Agent** - Manages enrollment queries and reports

In [32]:
# Agent 1: Course Information Agent
course_info_agent = LlmAgent(
    model=Gemini(model_id="gemini-2.0-flash-exp"),
    name="course_info_agent",
    instruction="""You are a helpful course information assistant at a college.
    
Your role is to:
- Help students discover available courses
- Provide detailed information about course content and requirements
- Answer questions about course topics, descriptions, and class levels
- Be friendly, informative, and encouraging

When a student asks about courses:
1. Use the list_available_courses tool to find relevant courses
2. Present the information in a clear, organized way
3. Highlight key details like course level (freshman/sophomore) and descriptions
4. Ask if they'd like more information or want to enroll

Do NOT enroll students - that's handled by the enrollment agent.""",
    tools=[list_available_courses]
)

print("✅ Course Information Agent created")

✅ Course Information Agent created


In [35]:
# Agent 2: Student Enrollment Agent
enrollment_agent = LlmAgent(
    model=Gemini(model_id="gemini-2.5-flash-lite"),
    name="enrollment_agent",
    instruction="""You are a friendly student enrollment assistant at a college.

Your role is to guide students through the enrollment process in an interactive, conversational way.

ENROLLMENT PROCESS:
1. When a student expresses interest in enrolling, first check if they've mentioned a specific course
2. If they haven't specified a course, use list_available_courses to show them options
3. Once they select a course, collect the following information:
   - First name
   - Last name
   - Student ID (a number)
   - Course ID they want to enroll in
4. BEFORE enrolling, confirm all details with the student:
   "Just to confirm: I'll enroll [First] [Last] (ID: [student_id]) in [course_name] (Course ID: [course_id]). Is this correct?"
5. Only after confirmation, use the enroll_student tool
6. Provide a clear confirmation message after successful enrollment

IMPORTANT GUIDELINES:
- Be conversational and friendly
- Ask for one piece of information at a time if needed
- Always confirm before finalizing enrollment
- If a student is already enrolled, inform them politely
- If they ask about courses, use list_available_courses to help them decide

Remember: You're here to make enrollment easy and stress-free!""",
    tools=[list_available_courses, enroll_student]
)

print("✅ Student Enrollment Agent created")

✅ Student Enrollment Agent created


In [37]:
# Agent 3: Administrator Agent
admin_agent = LlmAgent(
    model=Gemini(model_id="gemini-2.5-flash-lite"),
    name="admin_agent",
    instruction="""You are an administrative assistant for college enrollment management.

Your role is to help administrators access and analyze enrollment data.

CAPABILITIES:
- Search for students by name using search_students_by_name
- View all courses a specific student is enrolled in using get_student_enrollments
- View all students enrolled in a specific course using get_students_in_course
- List available courses using list_available_courses

GUIDELINES:
- Be professional and efficient
- Present data in a clear, organized format
- When searching, provide complete information including student IDs and course IDs
- If no results are found, suggest alternative search approaches
- Protect student privacy - only provide information when appropriately requested

You have access to enrollment records and course catalogs. How can I assist you today?""",
    tools=[
        search_students_by_name,
        get_student_enrollments,
        get_students_in_course,
        list_available_courses
    ]
)

print("✅ Administrator Agent created")

✅ Administrator Agent created


## Step 5: Create Session Service and Runners

Set up the infrastructure to run our agents.

In [ ]:
# Create session service for managing conversation state
session_service = InMemorySessionService()

In [99]:
course_info_runner = Runner(
    agent=course_info_agent,
    app_name= 'course_info_app',
    session_service=session_service
)

enrollment_runner = Runner(
    agent=enrollment_agent,
    app_name= 'enrollment_app',
    session_service=session_service
)

admin_runner = Runner(
    agent=admin_agent,
    app_name= 'admin_app',
    session_service=session_service
)

## Step 6: Test the Course Information Agent

Let's test the course information agent with a sample query.

In [120]:
USER_ID = 'test'
MODEL_NAME = 'Gemini'

In [122]:
await run_session(
    course_info_runner, 
    ["What Python courses do you offer?"],
    "initial-course-info-session",
)


 ### Session: initial-course-info-session

User > What Python courses do you offer?


Gemini >  We offer "Introduction To Python" (course id: 101) for freshmen. This course covers the basic concepts of Python and programming fundamentals.

Is there anything else I can help you with regarding this course, or would you like to look into other subjects?


## Step 7: Test the Enrollment Agent

Test the interactive enrollment process with a multi-turn conversation.

In [134]:
# Test 2: Enrollment Agent - Interactive Enrollment
print("\n" + "=" * 60)
print("TEST 2: Enrollment Agent - Interactive Enrollment")
print("=" * 60)

enrollment_session_id = 'enrollment_session'
enrollment_user_id = str(10002)

# Create the session first

try:
    session = await session_service.create_session(
        app_name="enrollment_app", user_id=enrollment_user_id, session_id=enrollment_session_id
    )
except:
    session = await session_service.get_session(
        app_name="enrollment_app", user_id=enrollment_user_id, session_id=enrollment_session_id
    )

# Conversation 1: Student asks about Python courses
msg1 = "I'm interested in learning Python. What courses are available?"
query = types.Content(role="user", parts=[types.Part(text=msg1)])

async for event in enrollment_runner.run_async(
    user_id=enrollment_user_id, session_id=session.id, new_message=query
):
    # Check if the event contains valid content
    if event.content and event.content.parts:
        # Filter out empty or "None" responses before printing
        if (
            event.content.parts[0].text != "None"
            and event.content.parts[0].text
        ):
            response1 = event.content.parts[0].text


TEST 2: Enrollment Agent - Interactive Enrollment


In [138]:
final_response1 = response1
print(f"\n🧑 Student: {msg1}")
print(f"\n🤖 Agent: {final_response1}")


🧑 Student: I'm interested in learning Python. What courses are available?

🤖 Agent: Great! We are offering "Introduction To Python" (Course ID: 101), which covers basic concepts of Python and programming basics.

Would you like to enroll in this course?


In [140]:
# Conversation 2: Student wants to enroll
msg2 = "I'd like to enroll in Introduction To Python"
query = types.Content(role="user", parts=[types.Part(text=msg2)])

async for event in enrollment_runner.run_async(
    user_id=enrollment_user_id, session_id=session.id, new_message=query
):
    # Check if the event contains valid content
    if event.content and event.content.parts:
        # Filter out empty or "None" responses before printing
        if (
            event.content.parts[0].text != "None"
            and event.content.parts[0].text
        ):
            response2 = event.content.parts[0].text
final_response2 = response2
print(f"\n🧑 Student: {msg2}")
print(f"\n🤖 Agent: {final_response2}")


🧑 Student: I'd like to enroll in Introduction To Python

🤖 Agent: Great! To enroll you in "Introduction To Python" (Course ID: 101), I'll need a few more details.

Could you please tell me your first name?


In [142]:
# Conversation 3: Provide student details
msg3 = "My name is Sarah Johnson and my student ID is 10002"
query = types.Content(role="user", parts=[types.Part(text=msg3)])

async for event in enrollment_runner.run_async(
    user_id=enrollment_user_id, session_id=session.id, new_message=query
):
    # Check if the event contains valid content
    if event.content and event.content.parts:
        # Filter out empty or "None" responses before printing
        if (
            event.content.parts[0].text != "None"
            and event.content.parts[0].text
        ):
            response3 = event.content.parts[0].text
final_response3 = response3
print(f"\n🧑 Student: {msg3}")
print(f"\n🤖 Agent: {final_response3}")


🧑 Student: My name is Sarah Johnson and my student ID is 10002

🤖 Agent: Just to confirm: I'll enroll Sarah Johnson (ID: 10002) in Introduction To Python (Course ID: 101). Is this correct?


In [144]:
# Conversation 4: Confirm enrollment
msg4 = "Yes, that's correct. Please enroll me."
query = types.Content(role="user", parts=[types.Part(text=msg4)])

async for event in enrollment_runner.run_async(
    user_id=enrollment_user_id, session_id=session.id, new_message=query
):
    # Check if the event contains valid content
    if event.content and event.content.parts:
        # Filter out empty or "None" responses before printing
        if (
            event.content.parts[0].text != "None"
            and event.content.parts[0].text
        ):
            response4 = event.content.parts[0].text
final_response4 = response4
print(f"\n🧑 Student: {msg4}")
print(f"\n🤖 Agent: {final_response4}")


🧑 Student: Yes, that's correct. Please enroll me.

🤖 Agent: Excellent! You've been successfully enrolled in "Introduction To Python" (Course ID: 101). Welcome aboard, Sarah!


## Step 8: Test the Administrator Agent

Test administrative queries for enrollment data.

In [152]:
# Test 3: Administrator Agent
print("\n" + "=" * 60)
print("TEST 3: Administrator Agent")
print("=" * 60)

admin_session_id = 'admin_session'
admin_user_id = 'admin_user'

try:
    admin_session = await session_service.create_session(
        app_name="admin_app", user_id=admin_user_id, session_id=admin_session_id
    )
except:
    admin_session = await session_service.get_session(
        app_name="admin_app", user_id=admin_user_id, session_id=admin_session_id
    )


TEST 3: Administrator Agent


In [164]:
# Query 1: Find students in a course
query1 = "Show me all students enrolled in course 101"
query = types.Content(role="user", parts=[types.Part(text=query1)])

async for event in admin_runner.run_async(
    user_id=admin_user_id, session_id=admin_session.id, new_message=query
):
    if event.content and event.content.parts:
        if (
            event.content.parts[0].text != "None"
            and event.content.parts[0].text
        ):
            admin_response1 = event.content.parts[0].text
    
print(f"\n👨‍💼 Admin: {query1}")
print(f"\n🤖 Agent: {admin_response1}")


👨‍💼 Admin: Show me all students enrolled in course 101

🤖 Agent: Here are the students enrolled in Course 101, Introduction To Python:

*   Sarah Johnson (ID: 10002)


In [168]:
# Query 2: Search for a student
query2 = "Search for students with the name Sarah"
query = types.Content(role="user", parts=[types.Part(text=query2)])

async for event in admin_runner.run_async(
    user_id=admin_user_id, session_id=admin_session.id, new_message=query
):
    if event.content and event.content.parts:
        if (
            event.content.parts[0].text != "None"
            and event.content.parts[0].text
        ):
            admin_response2 = event.content.parts[0].text
print(f"\n👨‍💼 Admin: {query2}")
print(f"\n🤖 Agent: {admin_response2}")


👨‍💼 Admin: Search for students with the name Sarah

🤖 Agent: Here are the students matching your search for "Sarah":

*   Sarah Johnson (ID: 10002)


In [170]:
# Query 3: Get a student's enrollments
query3 = "What courses is student 10002 enrolled in?"
query = types.Content(role="user", parts=[types.Part(text=query3)])

async for event in admin_runner.run_async(
    user_id=admin_user_id, session_id=admin_session.id, new_message=query
):
    if event.content and event.content.parts:
        if (
            event.content.parts[0].text != "None"
            and event.content.parts[0].text
        ):
            admin_response3 = event.content.parts[0].text
            
print(f"\n👨‍💼 Admin: {query3}")
print(f"\n🤖 Agent: {admin_response3}")


👨‍💼 Admin: What courses is student 10002 enrolled in?

🤖 Agent: Here are the courses student 10002 (Sarah Johnson) is enrolled in:

*   Introduction To Python (Course ID: 101)


## 🎓 College Enrollment System - Usage Guide

### Available Agents

1. **Course Information Agent** (`course_info_agent`)
   - Helps students discover and learn about courses
   - Provides course details, descriptions, and prerequisites
   - Example: "What machine learning courses do you offer?"

2. **Enrollment Agent** (`enrollment_agent`)
   - Guides students through interactive enrollment
   - Collects necessary information step-by-step
   - Confirms details before finalizing enrollment
   - Example: "I want to enroll in Introduction to Python"

3. **Administrator Agent** (`admin_agent`)
   - Helps administrators query enrollment data
   - Search students, view course rosters, check enrollments
   - Example: "Show me all students in course 101"


### Database Schema

**Courses Table:**
- `course_name` (TEXT): Name of the course
- `class` (TEXT): Class level (freshman/sophomore)
- `course_description` (TEXT): Course description
- `course_id` (INTEGER): Unique course identifier

**Enrollments Table:**
- `student_first_name` (TEXT): Student's first name
- `student_last_name` (TEXT): Student's last name
- `student_id` (INTEGER): Unique student identifier
- `course_name` (TEXT): Name of enrolled course
- `course_id` (INTEGER): ID of enrolled course

### Features

✅ Interactive, conversational enrollment process  
✅ Course discovery and information retrieval  
✅ Administrative enrollment management  
✅ Duplicate enrollment prevention  
✅ Confirmation before finalizing enrollments  
✅ Multi-turn conversations with context retention